"""

DATA ENGINEERING ETL PIPELINE - XETRA DATASET

4: Parametisation of data extraction.

Aim:
Write a production ready ETL pipeline using python and pandas.

Overview:
Xetra is a German stock exchange based in Frankfurt operated by Deutsche Börse Group. 
Data related to daily trading activity is stored publicly on the Amazon S3 database. 
(Update - as of July 2022 the data is no longer available. An archival S3 database will be used) 

Task:
Use jupyter notebook as a protoyping tool to extract and transform source data.
Request and extract source data from cloud based web services.
Use loops and iteration to read and consolidate multiple source files.
Familiarise with pandas package functions to clean and transform output data. 

Below outlines the steps to be performed:
    
    1) Continuation of xetra_3:
        - Transformation of data via sorting, grouping and aggregation. 
    2) Parametise filtering of xetra data bucket using datetime functions. 
    
    

    
"""

In [ ]:
arg_date = '2022-01-17' #Date argument to filter xetra data bucket.
arg_date_dt = datetime.strptime(arg_date) 

Data Extraction

In [350]:
import boto3 #AWS service management package.
import pandas as pd #Data analysis library.
from io import StringIO #String buffer to read CSV files.
from datetime import dateime, timedelta #Facilitate calulations relating to day of trade. 


In [351]:
s3 = boto3.resource('s3') #Use the Amazon S3 cloud storage resource.
bucket = s3.Bucket('xetra-1234') #Create instance of the "xetra" data bucket.

In [352]:
bucket_obj1 = bucket.objects.filter(Prefix='2022-01-28/') #Filter by date and store data as "bucket_obj1".
bucket_obj2 = bucket.objects.filter(Prefix='2022-02-28/') #Filter by date and store data as "bucket_obj2".
bucket_objects = [obj for obj in bucket_obj1] + [obj for obj in bucket_obj2]  #Store data into bucket list.

In [353]:
#Read csv body of dataset into pandas dataframe - initialisation step:
csv_obj_init = bucket.Object(key=bucket_objects[0].key).get().get('Body') #Initialise first element of csv object.
csv_obj_init = csv_obj_init.read().decode('utf-8') #Store into csv object in utf-8 format.
data = StringIO(csv_obj_init) #Convert csv object from streaming body to string data.
df_init = pd.read_csv(data, delimiter=',') #Read data into pandas data frame.
df_all = pd.DataFrame(columns=df_init.columns) #Initialise df_all with df_init columns.

In [354]:
#Read csv body of dataset into pandas dataframe - iteration step:
for obj in bucket_objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body') #Read data element from list.
    csv_obj = csv_obj.read().decode('utf-8') #Store into to csv object in utf-8 format.
    data = StringIO(csv_obj) #Convert csv object to string data.
    df = pd.read_csv(data, delimiter=',') #Read data as pandas data frame.
    df_all = pd.concat([df, df_all]) #Concatenate data to one master dataframe.

In [355]:
csv_obj #Print csv object to view columns.

'ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades\r\n'

Data Cleansing

In [356]:
#Remove unecessary columns by storing required columns in variable and passing as .loc function argument. 
columns_use = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:,columns_use]

In [357]:
df_all.dropna(inplace=True) #Drop all missing values from the dataset.
df_all = df_all.reset_index(drop=True) #Reset the column index.
df_all.shape #Check if there was any filtering (should match table dimensions)

(257248, 8)

Data Transformation

In [358]:
#Get opening price per ISIN on a particular day.  
df_all['OpeningPrice'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform('first')

#Get closing price per ISIN on a particular day. 
df_all['ClosingPrice'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['EndPrice'].transform('last')

#Aggregate data per ISIN on a particular day.
df_all = df_all.groupby(['ISIN','Date'], as_index = False).agg(OpeningPriceEUR = ('OpeningPrice', 'min'),ClosingPriceEUR = ('ClosingPrice', 'min'), MinPriceEUR = ('MinPrice', 'min'), MaxPriceEUR = ('MaxPrice', 'max'), DailyTradedVolume = ('TradedVolume', 'sum'))

#Percentage change in closing price between current and pervious day of trade. 
df_all['PrevClosingPriceEUR'] = df_all.sort_values(by = 'Date').groupby(['ISIN'])['ClosingPriceEUR'].shift(1)
df_all['DeltaPCP%'] = (df_all['ClosingPriceEUR'] - df_all['PrevClosingPriceEUR'])/df_all['PrevClosingPriceEUR']*100

#Cleanse aggregated data. 
df_all.drop(columns = ['PrevClosingPriceEUR'], inplace = True) 
df_all = df_all.round(decimals = 2)


In [359]:
#Print transformed data output.
df_all

,ISIN,Date,OpeningPriceEUR,ClosingPriceEUR,MinPriceEUR,MaxPriceEUR,DailyTradedVolume,DeltaPCP%
0,AT000000STR1,2022-01-28,38.05,37.00,37.00,38.05,456,NaN
1,AT000000STR1,2022-02-28,36.60,36.70,35.75,36.70,1773,-0.81
2,AT00000FACC2,2022-01-28,7.66,7.52,7.52,7.66,610,NaN
3,AT00000FACC2,2022-02-28,8.05,8.57,7.87,8.57,10205,13.96
4,AT0000606306,2022-01-28,25.02,24.66,24.66,25.10,213,NaN
...,...,...,...,...,...,...,...,...
6395,XS2314660700,2022-01-28,20.33,20.25,20.05,20.33,58,NaN
6396,XS2314660700,2022-02-28,22.26,21.92,21.92,22.28,0,8.26
6397,XS2376095068,2022-01-28,32.99,33.23,32.77,33.23,0,NaN
6398,XS2376095068,2022-02-28,34.29,36.50,34.06,36.50,1000,9.84
